In [52]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import json
import pandas as pd
import re
import seaborn as sns
import datetime
from datetime import timedelta
import numpy as np

In [53]:
airport_code = ['LAS', 'LAX', 'CHI', 'SFO', 'ORL', 'DEN', 'DCA', 'SEA', 'BOS']

day_delta = datetime.timedelta(days=1)
start_date = datetime.date(2018, 12, 22)
end_date = start_date + 11*day_delta

url_list = []
for i in range((end_date - start_date).days):
    day = start_date + i*day_delta
    for airport in airport_code: 
        url = 'https://www.kayak.com/flights/NYC-' + str(airport) + '/' + str(day) + '?sort=bestflight_a&fs=stops=0'
        url_list.append(url)

In [54]:
def create_flight_price_text(flight_url):
    r = requests.get('https://www.kayak.com')
    layer1Cookies = r.cookies

    flight_search_xhr = 'https://www.kayak.com/s/horizon/flights/results/FlightSearchPoll'

    params1 = {
        'searchId':'',
        'poll':'true',
        'pollNumber':'0',
        'applyFilters':'true',
        'filterState':'',
        'useViewStateFilterState':'false',
        'pageNumber':'1',
        'append':'false',
        'pollingId':'593601',  #interesting. explore further
        'requestReason':'POLL',
        'isSecondPhase':'false',
        'textAndPageLocations':'bottom,right',
        'displayAdPageLocations':'none',
        'existingAds':'false',
        'activeLeg':'-1',
        'view':'list',
        'renderPlusMinusThreeFlex':'false',
        'renderAirlineStopsMatrix':'false',
        'renderFlexHeader':'true',
        'tab':'flights',
        'pageOrigin':'F..FD..M0',
        'src':'',
        'searchingAgain':'',
        'c2s':'',
        'po':'',
        'personality':'',
        'provider':'',
        'isMulticity':'false',
        'flex_category':'exact',
        'oneway':'false',
        'nearby_origin':'false',
        'nearby_destination':'false',
        'countrySearch':'false',
        'travelers':'1',
        'adults':'1',
        'seniors':'0',
        'youth':'0',
        'child':'0',
        'seatInfant':'0',
        'lapInfant':'0',
        'cabin':'e',
        'cabinDisplayType':'Economy',
        'vertical':'flights',
        'url':flight_url,
        'id':'',
        'navigateToResults':'false',
        'ajaxts':'',
        'scriptsMetadata':'',
        'stylesMetadata':'',
    }
    
    headers = {
        'Host': 'www.kayak.com',
        'User-Agent': 'Chrome/63 (Macintosh; Intel Mac OS X 10.11; rv:57.0) Gecko/20100101 Firefox/57.0',
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': flight_url,
        'Content-Type': 'application/x-www-form-urlencoded',
        'X-CSRF': 'kAqI1NgGh$DJnEUpiSDOWpdQXzlgAwG8EVOCd$gXO08-hpumC4oNpaOjz15GO_q9a5FdZPonpC2kF4CBYjEPh14',
        'X-RequestId': 'flights#frontdoor#Ag$s9g',
        'X-Requested-With': 'XMLHttpRequest',
        'Content-Length': '1094'
    }


    result = requests.post(flight_search_xhr, headers = headers, data = params1, cookies = layer1Cookies)
    text = result.json()
    text = BeautifulSoup(text['content'], 'html.parser')
    return text;

In [55]:
def getPricePrediction(flight_url):
    prediction_url = "https://www.kayak.com/s/horizon/flights/results/FlightPricePredictionAction"
    flight_search_html = requests.get(flight_url)

    s_id = re.compile(r'searchID=(\w+)')
    matches = s_id.finditer(str(create_flight_price_text(url)))
    searchId = ""
    for m in matches:
        searchId = m.group(1)

    token = re.compile(r"""\"formtoken\":\"(.+)\"\,""")
    matches = token.finditer(flight_search_html.text)
    formtoken = ""
    for m in matches:
        formtoken = m.group(1)

    params = {
        'searchId': searchId,
        'formtoken': formtoken
    }
    
    headers = {
        'Host': 'www.kayak.com',
        'User-Agent': 'Chrome/63 (Macintosh; Intel Mac OS X 10.11; rv:57.0) Gecko/20100101 Firefox/57.0',
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': flight_url,
        'Content-Type': 'application/x-www-form-urlencoded',
        'X-CSRF': 'kAqI1NgGh$DJnEUpiSDOWpdQXzlgAwG8EVOCd$gXO08-hpumC4oNpaOjz15GO_q9a5FdZPonpC2kF4CBYjEPh14',
        'X-RequestId': 'flights#frontdoor#Ag$s9g',
        'X-Requested-With': 'XMLHttpRequest',
        'Content-Length': '1094'
    }

    
    advice = requests.post(prediction_url, headers=headers, data=params)
    print(advice.json())

In [ ]:
for url in url_list:
    getPricePrediction(url)

{'infoText': "Prices predicted to rise <br> <span class='price'>$16</span> by <span class='date'>Dec 4</span>", 'cssClass': 'buy', 'advice': 'buy', 'moreInfoText': "Our data scientists think these are the best prices you'll see for the next seven days.\nLike weather forecasters, though, they can't be 100% certain. Our purchase advisory, shown here, is based on an analysis of current and past prices.", 'buy': True, 'actionableAmountMsg': 'Prices will likely increase $16', 'adviceText': 'Our Advice: Buy', 'alertsCta': 'Track price changes with our Price Alert emails.', 'actionableDateMsg': 'Within 7 days'}
{'infoText': "Prices predicted to rise <br> <span class='price'>$19</span> by <span class='date'>Dec 4</span>", 'cssClass': 'buy', 'advice': 'buy', 'moreInfoText': "Our data scientists think these are the best prices you'll see for the next seven days.\nLike weather forecasters, though, they can't be 100% certain. Our purchase advisory, shown here, is based on an analysis of current an

In [32]:
def get_prices_times(text):
    doc = html.fromstring(str(text))
    flights = doc.xpath('//div[@class="Base-Results-HorizonResult Flights-Results-FlightResultItem phoenix-rising sleek rp-contrast "]')
    
    result = []
    for f in flights:
        price_info = f.get('aria-label')
        time_info = f.xpath('.//div//div//div//div[@class="col-info result-column"]')[0]
        time = time_info.xpath('.//div//div//ol//li//div//div//div//div//span//span[@class="depart-time base-time"]')[0].text
        am_pm = time_info.xpath('.//div//div//ol//li//div//div//div//div//span//span[@class="time-meridiem meridiem"]')[0].text
        
        regex_price = re.compile(r'(\$)(\d*)',re.VERBOSE)
        matches_price = regex_price.finditer(price_info)
        for match in matches_price:
            clean_price = match.group(2)
            entry = {
                'price' : clean_price,
                'time' : time + am_pm
            }
            result.append(entry)
        
    return result

In [33]:
def get_flights(html):
    flights = html.find_all('li', 'flight')
    result = []
    for f in flights:
        info = str(f.find('div', 'bottom'))
        info = info.replace('<div class="bottom">', '')
        info = info.replace("</div>", '')
        result.append(info)
        
    return result

In [50]:
for url in url_list:
#     print(type(url))
    print(url)
    print(get_prices_times(create_flight_price_text(url)))
    print('\n')

https://www.kayak.com/flights/NYC-LAS/2018-12-22?sort=bestflight_a&fs=stops=0
[{'price': '313', 'time': '2:40 pm'}, {'price': '379', 'time': '7:55 pm'}, {'price': '382', 'time': '9:16 pm'}, {'price': '317', 'time': '10:15 am'}, {'price': '317', 'time': '7:20 am'}, {'price': '327', 'time': '2:40 pm'}, {'price': '382', 'time': '5:00 pm'}, {'price': '389', 'time': '8:35 pm'}, {'price': '398', 'time': '8:45 pm'}, {'price': '404', 'time': '5:18 pm'}, {'price': '411', 'time': '8:35 am'}, {'price': '413', 'time': '5:59 pm'}, {'price': '431', 'time': '4:49 pm'}, {'price': '435', 'time': '8:45 pm'}, {'price': '437', 'time': '11:52 am'}]


https://www.kayak.com/flights/NYC-LAX/2018-12-22?sort=bestflight_a&fs=stops=0
[{'price': '374', 'time': '10:15 am'}, {'price': '491', 'time': '4:40 pm'}, {'price': '538', 'time': '9:15 pm'}, {'price': '380', 'time': '2:40 pm'}, {'price': '413', 'time': '10:50 pm'}, {'price': '413', 'time': '5:59 pm'}, {'price': '443', 'time': '9:59 pm'}, {'price': '452', 'time

KeyboardInterrupt: 

In [47]:
get_flights(text)

['JetBlue',
 'United Airlines',
 'Delta',
 'JetBlue',
 'Spirit Airlines',
 'Frontier',
 'Frontier',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines']